In [4]:
%env TOKENIZERS_PARALLELISM=false



env: TOKENIZERS_PARALLELISM=false


In [5]:
!pip install ipywidgets
!jupyter labextension install @jupyter-widgets/jupyterlab-manager




usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: bundlerextension console dejavu execute kernel
kernelgateway kernelspec migrate nbclassic nbconvert nbextension notebook run
server serverextension troubleshoot trust

Jupyter command `jupyter-labextension` not found.


In [6]:
!pip install -U sentence-transformers
!pip install tf-keras
!pip install ipywidgets
from sklearn.preprocessing import StandardScaler
import ast
import pickle

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sentence_transformers import SentenceTransformer
import re

In [8]:
df= pd.read_csv('products_train.csv')
products_df=df

In [9]:
products_df.head()



,id,locale,title,price,brand,color,size,model,material,author,desc
0,B005ZSSN10,DE,RED DRAGON Amberjack 3 - Steel Tip 22 Gramm Wo...,30.95,RED DRAGON,NaN,NaN,RDD0089,NaN,NaN,Amberjacks Steel Dartpfeile sind verfügbar in ...
1,B08PRYN6LD,DE,Simply Keto Lower Carb* Schokodrops ohne Zucke...,17.90,Simply Keto,NaN,750 g (1er Pack),NaN,NaN,NaN,🌱 NATÜRLICHE SÜSSE DURCH ERYTHRIT - Wir stelle...
2,B09MBZJ48V,DE,"Sennheiser 508377 PC 5.2 Chat, Stilvolles Mult...",68.89,Sennheiser,Multi-Colour,One size,508377,Kunstleder,NaN,3.5 MM BUCHSE - Kann problemlos an Geräte mit ...
3,B08ZN6F26S,DE,AmyBenton Auto ab 1 2 3 ahre - Baby Aufziehbar...,18.99,Amy & Benton,Animal Car,NaN,2008B,aufziehauto 1 jahr,NaN,【Auto aufziehbar】: Drücken Sie einfach leicht ...
4,B094DGRV7D,DE,PLAYMOBIL - 70522 - Cavaliere mit grauem Pony,7.17,PLAYMOBIL,Nicht Zutreffend.,OneSize,70522,Polypropylen,NaN,Inhalt: 1 Stück


In [10]:
sess_df= pd.read_csv("sessions_train.csv")

In [11]:
sess_df.head()


,prev_items,next_item,locale
0,['B09W9FND7K' 'B09JSPLN1M'],B09M7GY217,DE
1,['B076THCGSG' 'B007MO8IME' 'B08MF65MLV' 'B001B...,B001B4THSA,DE
2,['B0B1LGXWDS' 'B00AZYORS2' 'B0B1LGXWDS' 'B00AZ...,B0767DTG2Q,DE
3,['B09XMTWDVT' 'B0B4MZZ8MB' 'B0B7HZ2GWX' 'B09XM...,B0B4R9NN4B,DE
4,['B09Y5CSL3T' 'B09Y5DPTXN' 'B09FKD61R8'],B0BGVBKWGZ,DE


In [12]:
df_small = products_df.sample(n=200000, random_state=42)


In [13]:
cols = ['title', 'brand', 'desc', 'material','color','author']

In [14]:
cols

['title', 'brand', 'desc', 'material', 'color', 'author']

In [15]:
#filling empty spaces with space' '
df_small.loc[:, cols] = df_small.loc[:, cols].fillna('')


In [16]:
# Combine the selected columns into a single string per row
df_small['text_combined'] = (
    "title: " + df_small['title'].fillna('') + " " +
    "brand: " + df_small['brand'].fillna('') + " " +
    "desc: " + df_small['desc'].fillna('') + " " +
    "material: " + df_small['material'].fillna('') + " " +
    "color: " + df_small['color'].fillna('') + " " +
    "author: " + df_small['author'].fillna('')
)

df_small['text_combined'].head()

,text_combined
781974,title: kashiwise ディスプレイケース カードスタンド アクリルフレーム br...
937737,title: Blumomon 1 Pack Soccer Cake Topper Happ...
907828,title: ポーラ B.A アイゾーンクリーム N 26g ＜【医薬部外品】リンクルショッ...
784628,title: ニュートロ シュプレモ ドッグフード 超小型犬~小型犬用 成犬用 プレミアムブ...
662460,title: SKYN コンドーム 10個入ラージサイズ brand: 不二ラテックス de...


In [17]:
print(df_small.iloc[1]['text_combined'])

title: Blumomon 1 Pack Soccer Cake Topper Happy Birthday Sign Football Player Cake Decorations for Sport Theme Man Boy Girl Birthday Party Decoration Supplies brand: Blumomon desc: ★Material: High quality 300g cardboard with food grade toothpicks sticks, attractive and durable！ material: Paper color: Green author: 


In [18]:
# converting everything to lower case
df_small.loc[:,'text_combined'] = df_small['text_combined'].str.lower()


In [19]:
print(df_small.iloc[1]['text_combined'])

title: blumomon 1 pack soccer cake topper happy birthday sign football player cake decorations for sport theme man boy girl birthday party decoration supplies brand: blumomon desc: ★material: high quality 300g cardboard with food grade toothpicks sticks, attractive and durable！ material: paper color: green author: 


In [20]:
#function to remove emojis
def remove_emoji(text):
    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"  # emoticons
        "\U0001F300-\U0001F5FF"  # symbols & pictographs
        "\U0001F680-\U0001F6FF"  # transport & map symbols
        "\U0001F1E0-\U0001F1FF"  # flags
        "\U00002702-\U000027B0"  # dingbats
        "\U000024C2-\U0001F251"
        "]+",
        flags=re.UNICODE,
    )
    return emoji_pattern.sub(r'', text)

In [21]:
# removing emojis
df_small.loc[:,'text_combined'] = df_small['text_combined'].apply(remove_emoji)

In [22]:
print(df_small.iloc[1]['text_combined'])

title: blumomon 1 pack soccer cake topper happy birthday sign football player cake decorations for sport theme man boy girl birthday party decoration supplies brand: blumomon desc: material: high quality 300g cardboard with food grade toothpicks sticks, attractive and durable material: paper color: green author: 


In [23]:
df_small['text_combined'].isnull().sum()


np.int64(0)

In [29]:
import ast

def safe_parse_list(val):
    try:
        parsed = ast.literal_eval(val)
        return parsed if isinstance(parsed, list) else []
    except:
        return []

# Apply with tqdm to monitor progress
from tqdm import tqdm
tqdm.pandas()

sess_df['prev_items'] = sess_df['prev_items'].progress_apply(safe_parse_list)


100%|██████████| 3606249/3606249 [00:20<00:00, 174913.18it/s]


In [30]:
# Step 1: Allowed product IDs set
allowed_product_ids = set(df_small['id'])

# Step 2: Function to check if all products in a session are valid
def is_valid_session(row):
    all_items = set(row['prev_items'] + [row['next_item']])
    return all_items.issubset(allowed_product_ids)

# Step 3: Filter sessions
filtered_sess_df = sess_df[sess_df.progress_apply(is_valid_session, axis=1)]

# Step 4: Display how many valid sessions we got
print("Filtered session count:", len(filtered_sess_df))




100%|██████████| 3606249/3606249 [00:29<00:00, 121357.18it/s]

Filtered session count: 559241


In [28]:
model = SentenceTransformer('distiluse-base-multilingual-cased-v2')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.46k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/model.safetensors:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

2_Dense/pytorch_model.bin:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

In [29]:
#sample_embeddings = model.encode(sample_df['text_combined'].tolist(), show_progress_bar=True)

In [30]:
#print(sample_embeddings[0])          # First product's embedding
#print(sample_embeddings.shape)       # Should be (100, 512)


In [31]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [32]:
from tqdm import tqdm
import numpy as np

texts = df_small['text_combined'].tolist()
batch_size = 10000
all_embeddings = []

for i in tqdm(range(0, len(texts), batch_size)):
    batch_texts = texts[i:i+batch_size]
    batch_embeddings = model.encode(batch_texts, batch_size=32, show_progress_bar=False)

    # Save each batch to disk in the correct folder
    np.save(f'/content/drive/MyDrive/Colab Notebooks/embeddings_batch_{i//batch_size}.npy', batch_embeddings)
    all_embeddings.append(batch_embeddings)



100%|██████████| 20/20 [07:52<00:00, 23.63s/it]


In [33]:
import glob
import numpy as np

# List and load all batch files from the Colab Notebooks folder
files = sorted(glob.glob('/content/drive/MyDrive/Colab Notebooks/embeddings_batch_*.npy'))
all_batches = [np.load(f) for f in files]

# Stack into one full array
final_embeddings = np.vstack(all_batches)

# Save the final array to Drive (also inside Colab Notebooks folder)
np.save('/content/drive/MyDrive/Colab Notebooks/final_embeddings.npy', final_embeddings)


In [32]:
import numpy as np
import pandas as pd

# Re-load
final_embeddings = np.load('/content/drive/MyDrive/Colab Notebooks/final_embeddings.npy')



In [33]:
n = min(100_000, len(filtered_sess_df))
sess_df_100k = filtered_sess_df.sample(n=n, random_state=42).reset_index(drop=True)

print("Sampled sessions:", len(sess_df_100k))


Sampled sessions: 100000


In [34]:
# Assuming 'final_embeddings' is your numpy array of shape (200000, embedding_dim)
# and 'df_small' has a column 'id' in the same order as embeddings

import numpy as np

product_ids = df_small['id'].tolist()

id_to_embedding = {
    product_id: final_embeddings[i]
    for i, product_id in enumerate(product_ids)
}


In [35]:
import torch

X = []  # input sequences
y = []  # next item embeddings (targets)

for _, row in sess_df_100k.iterrows():
    prev_ids = row['prev_items']
    next_id = row['next_item']

    if all(pid in id_to_embedding for pid in prev_ids) and next_id in id_to_embedding:
        sequence = [id_to_embedding[pid] for pid in prev_ids]
        target = id_to_embedding[next_id]

        X.append(sequence)
        y.append(target)


In [36]:
from torch.nn.utils.rnn import pad_sequence
import torch

# Convert each list of embeddings into a torch.Tensor
X_tensors = [torch.tensor(seq, dtype=torch.float32) for seq in X]
y_tensors = torch.tensor(y, dtype=torch.float32)

# Pad sequences to the same length
X_padded = pad_sequence(X_tensors, batch_first=True)

print("X shape (padded):", X_padded.shape)
print("y shape:", y_tensors.shape)


<ipython-input-36-2809973812>:6: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  y_tensors = torch.tensor(y, dtype=torch.float32)


X shape (padded): torch.Size([100000, 0])
y shape: torch.Size([100000, 512])


In [37]:
import torch.nn as nn
import torch.nn.functional as F

class LSTMAttentionModel(nn.Module):
    def __init__(self, embedding_dim, hidden_dim):
        super(LSTMAttentionModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.attention = nn.Linear(hidden_dim, 1)
        self.fc = nn.Linear(hidden_dim, embedding_dim)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)  # x: (batch_size, seq_len, embedding_dim)

        # Attention weights
        attn_weights = F.softmax(self.attention(lstm_out), dim=1)  # (batch, seq_len, 1)
        context = torch.sum(attn_weights * lstm_out, dim=1)        # (batch, hidden_dim)

        out = self.fc(context)  # (batch, embedding_dim)
        return out


In [38]:
import torch
from torch.utils.data import Dataset

class SessionDataset(Dataset):
    def __init__(self, session_df, id_to_embedding, max_len=10):
        self.samples = []
        self.embedding_dim = len(next(iter(id_to_embedding.values())))
        self.max_len = max_len

        for _, row in session_df.iterrows():
            prev_ids = row['prev_items'][-max_len:]  # clip long sequences
            padded = [id_to_embedding[i] for i in prev_ids]

            # pad if needed
            while len(padded) < max_len:
                padded.insert(0, np.zeros(self.embedding_dim))

            next_emb = id_to_embedding.get(row['next_item'])
            if next_emb is not None:
                self.samples.append((np.stack(padded), np.array(next_emb)))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        x, y = self.samples[idx]
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)


In [39]:
from torch.utils.data import DataLoader

# Use the correct session DataFrame
dataset = SessionDataset(sess_df_100k, id_to_embedding, max_len=10)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

# Define model, loss, and optimizer
embedding_dim = next(iter(id_to_embedding.values())).shape[0]
hidden_dim = 128

model = LSTMAttentionModel(embedding_dim, hidden_dim)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)


In [40]:
num_epochs = 5  # Start with 5 epochs (you can increase later)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0

    for batch in dataloader:
        X_batch, y_batch = batch
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        optimizer.zero_grad()
        outputs = model(X_batch)

        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    print(f"Epoch [{epoch+1}/{num_epochs}] - Loss: {avg_loss:.4f}")



Epoch [1/5] - Loss: 0.0009
Epoch [2/5] - Loss: 0.0009
Epoch [3/5] - Loss: 0.0009
Epoch [4/5] - Loss: 0.0009
Epoch [5/5] - Loss: 0.0009


In [41]:
# Save model weights
torch.save(model.state_dict(), "/content/drive/MyDrive/Colab Notebooks/lstm_attention_model.pth")
print("Model saved successfully.")


Model saved successfully.
